In [ ]:
import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import re
import squarify

from functions import run_proportional_scaling, run_linear_programming, save_data_to_time_series
from helpers import fetch_area_data, get_variables
from tree import build_tree, fetch_branch, fetch_values_given_key, get_subindustries_data, get_constraints, write_into
from config import settings

In [ ]:
print('51163 Rockbridge County, Virginia')
df, url = fetch_area_data('2021',4,'51163')

In [ ]:
# https://www.bls.gov/cew/classifications/aggregation/agg-level-titles.htm
total_establishments = int(df[df['agglvl_code']==70][settings.establishments])
print(f"Total establishments in the county: {total_establishments}") 
aggregations = range(74,78+1)
for aggregation in aggregations:
    print(f"The sum of establishment counts at level {aggregation}: {np.sum(df[df['agglvl_code']==aggregation][settings.establishments])}")

In [ ]:
# cast data into a tree structure with nested mappings
# county = {'ind': 10, 'est': 533, 'emp': 5937, children': [{}, {}, ...]}
county = build_tree(df, '10', 71)

In [ ]:
# Return industry details
industry = fetch_branch(county, 'ind', '62')
sub_industries = [sub_industry['ind'] for sub_industry in industry['children']]
est = [sub_industry['est'] for sub_industry in industry['children']]

sub_industries_sorted = [ind for _, ind in sorted(zip(est, sub_industries), reverse=True)]
est_sorted = sorted(est, reverse=True)
squarify.plot(sizes=est_sorted, label=sub_industries_sorted)
plt.axis('off')
plt.show()

In [ ]:
# Proportional scaling example
industry = fetch_branch(county, 'ind', '10')
print('*** overall number of establishments ***')
print(industry['est'])
print('*** overall employment ***')
print(industry['emp'])
print('*** sub industries ***')
print([sub_industry['ind'] for sub_industry in industry['children']])
print('*** number of establishments in the sub industries ***')
print([sub_industry['est'] for sub_industry in industry['children']])
print('*** employment in the sub industries ***')
print([sub_industry['emp'] for sub_industry in industry['children']])
print('*** employment approximation proportional scaling ***')
establishments = np.array([sub_industry['est'] for sub_industry in industry['children']])
employment = np.array([sub_industry['emp'] for sub_industry in industry['children']],dtype = object)
employment_ps = employment.copy()
undisclosed_est = np.sum(establishments[np.where(employment==0)])
undisclosed_emp = industry['emp'] - np.sum(employment)
employment_ps[np.where(employment==0)] = establishments[np.where(employment==0)]/undisclosed_est*undisclosed_emp
print(employment_ps)
print('*** wages in the sub industries ***')
print([sub_industry['wages'] for sub_industry in industry['children']])
print('*** wages approximation proportional scaling ***')
establishments = np.array([sub_industry['est'] for sub_industry in industry['children']])
wages = np.array([sub_industry['wages'] for sub_industry in industry['children']],dtype = object)
wages_ps = wages.copy()
undisclosed_est = np.sum(establishments[np.where(wages==0)])
undisclosed_wages = industry['wages'] - np.sum(wages)
wages_ps[np.where(wages==0)] = establishments[np.where(wages==0)]/undisclosed_est*undisclosed_wages
print(wages_ps)



In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        y = employment,
        name = 'employment'
    )
)
fig.add_trace(
    go.Bar(
        y = employment_ps,
        name = 'employment ps'
    )
)
fig.update_layout(
    title='Employment and employment approximation',
    width=800,
    height=450,
    legend=dict(
                y=0.99,
                x=0.01
                )
)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        y = wages,
        name = 'wages'
    )
)
fig.add_trace(
    go.Bar(
        y = wages_ps,
        name = 'wages ps'
    )
)
fig.update_layout(
    title='Wages and wages approximation',
    width=800,
    height=450,
    legend=dict(
                y=0.99,
                x=0.01
                )
)

In [ ]:
# loop through the entire tree
for ind in fetch_values_given_key(county, 'ind', []):
    industry = fetch_branch(county, 'ind', ind)
    data = get_subindustries_data(industry)
    county = run_proportional_scaling(county, industry, data)

In [ ]:
# Define constraints
constraints = get_constraints(county, 'emp', [])

# Define variables
variables = []
for constraint in constraints:
    variables+=re.findall(r"epe_[^ ]* ", constraint)
for variable in list(np.unique(variables)):
    exec(f"{variable} = cp.Variable()")

# Define the objective function
exec(f"objective = cp.Minimize(cp.abs({constraints[0].replace('=', '-').replace('+', '-')}))")

# Define numerical constraints
numerical_constraints = []
for i,constraint in enumerate(constraints):
    if i > 0:
        numerical_constraints.append(eval(f"{constraint.replace('=','==')}"))
# Adding non-negativity constraints for the variables
for variable in list(np.unique(variables)):
    numerical_constraints.append(eval(f"{variable}>= 0"))

# Form and solve the problem
problem = cp.Problem(objective, numerical_constraints)
problem.solve()

In [ ]:
for variable in variables:
    ind = variable[variable.find('_')+1:-1]
    branch = fetch_branch(county, 'ind', ind)
    if branch['emp'] == 0:
        write_into(
            county, 'ind', branch['ind'],
            {'emp_lp': branch['est'] * eval(f"float({variable}.value)")}
        )


In [ ]:
industry = fetch_branch(county, 'ind', '31_33')
print(industry['est'])
print(industry['emp'])
if industry.get('emp_ps') is not None:
    print(f"emp_ps {industry['emp_ps']}")
if industry.get('emp_lp') is not None:
    print(f"emp_lp {industry['emp_lp']}")
for child in industry['children']:
    print(f"*** ind {child['ind']} *** ")
    print(child['est'])
    print(child['emp'])
    if child.get('emp_ps') is not None:
        print(f"emp_ps {child['emp_ps']}")
    if child.get('emp_lp') is not None:
        print(f"emp_lp {child['emp_lp']}")

In [ ]:
# Fetch all industry codes in a period
county_code = '51163'
years = list(range(2014,2021+1))
industry_codes = []
for year in years:
    df, url = fetch_area_data(year,4,county_code)
    county = build_tree(df, '10', 71)
    industry_codes+=fetch_values_given_key(county, 'ind', [])
time_series = pd.DataFrame([], columns=np.unique(industry_codes))

In [ ]:
# Run LP on all industries and all the years and save to time_series
for year in years:
    df, url = fetch_area_data(year,4,county_code)
    county = build_tree(df, '10', 71)
    county = run_linear_programming(county, 'emp')
    for ind in industry_codes:
        industry = fetch_branch(county, 'ind', ind)
        if industry is not None:
            if industry.get('emp_lp') is not None:
                time_series.at[year, ind] = industry['emp_lp']
            else:
                time_series.at[year, ind] = industry['emp']

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x = years,
        y = time_series['5221']
    )
)
fig.update_layout(
    title='Employment approximation with LP',
    width=800,
    height=450,
    legend=dict(
                y=0.99,
                x=0.01
                )
)

In [ ]:
time_series.pct_change().std().sort_values(ascending=False)[0:20]

In [ ]:
time_series.to_csv('Rockbridge.csv')

In [ ]:
# Fetch all industry codes in a period
county_code = '13121'
years = list(range(2014,2021+1))
industry_codes = []
for year in years:
    df, url = fetch_area_data(year,4,county_code)
    county = build_tree(df, '10', 71)
    industry_codes+=fetch_values_given_key(county, 'ind', [])
employment_series = pd.DataFrame([], columns=np.unique(industry_codes))
wages_series = pd.DataFrame([], columns=np.unique(industry_codes))


In [ ]:
# Run LP on all industries and all the years and save to time_series
for year in years:
    df, url = fetch_area_data(year,4,county_code)
    county = build_tree(df, '10', 71)
    county = run_linear_programming(county, 'emp')
    county = run_linear_programming(county, 'wages')
    employment_series = save_data_to_time_series(
        employment_series, county, industry_codes, 'emp', year
    )    
    wages_series = save_data_to_time_series(
        wages_series, county, industry_codes, 'wages', year
    )

In [ ]:
employment_series

In [ ]:
employment_series.pct_change().std().sort_values(ascending=False)[0:20]

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = years,
        y = employment_series['713950'],
        name = 'disclosed'
    )
)
fig.add_trace(
    go.Scatter(
        x = years[-2:],
        y = employment_series['713950'].iloc[-2:],
        name = 'undisclosed'

    )
)
fig.update_layout(
    title='Employment in Bowling Centers (713950), Fulton Georgia',
    width=800,
    height=450,
    legend=dict(
                y=0.99,
                x=0.01
                )
)

In [ ]:
employment_series.columns = employment_series.columns.str.replace('_', '-')
employment_series.to_csv('Fulton_employment.csv')

In [ ]:
wages_series.pct_change().std().sort_values(ascending=False)[0:20]

In [ ]:
wages_series.columns = wages_series.columns.str.replace('_', '-')
wages_series.to_csv('Fulton_wages.csv')